# Добро пожаловать!

Этот блокнот предназначен для парсинга и обработки информации с сайта [dom.mingkh.ru](https://dom.mingkh.ru/). 

Впоследствии подготовленные данные можно загрузить в QGIS и геокодировать (= присвоить координаты) адреса с помощью встроенной функции «Пакетный геокодер Nominatim».

Проверить корректность написания адреса можно на сайте [Nominatim OpenStreetMap](https://nominatim.openstreetmap.org/ui/search.html).



# Шаг 1. Импортируем библиотеки

In [ ]:
# Шаг 1. Импортируем нужные библиотеки

import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import numpy as np
from tqdm import tqdm

# Шаг 2. Инициализируем функцию

Нам нужна функция, которая пройдёт по веб-таблице со всеми жилыми домами города N, соберёт ссылки на страницы с детальной информацией про дома и вернёт датафрейм с подробной информацией.

В приведённой ниже функции мы собираем:

    1. Номер (порядковый в системе dom.mingkh для конкретного населённого пункта), 
    2. Адрес (дома), 
    3. Ссылка (на детальное описание дома), 
    4. Площадь всего (жилые, нежилые помещения и общее имущество), 
    5. Год постройки (дома), 
    6. Число этажей,
    7. Аварийный (ли дом, ответы «да» или «нет», 
    8. Состояние дома (исправный или аварийный)
    9. Количество квартир (в доме), 
    10. Тип дома («многоквартирный дом» или нечто иное),
    11. Площадь жилых (помещений) в м2, 
    12. Площадь нежилых (помещений) в м2', 
    13. Культурное наследие (является ли дом объектом культурного наследия).
    
Однако на страницах домов содержится больше информации — вы можете добавить её вручную, самостоятельно поработав с кодом по нашему примеру.

Примеры: 

* [таблица всех жилых домов Москвы](https://dom.mingkh.ru/moskva/moskva/houses), 
* [подробные данные о конкретном доме](https://dom.mingkh.ru/moskva/moskva/716860).

In [ ]:
# Шаг 2. Инициализируем нужные функции

def get_a_df_from_one_page (initial_table):
    
    ''' Эта функция принимает на вход html-таблицу и возвращает pandas-датафрейм со собранными атрибутами. 
    Если страница дома получена успешно, но данных нет, в таблицу добавится значение "Нет информации". 
    Если же код по какой-то причине сработает с ошибкой, будут выведены сообщения "Nothing to show" и "No data".
    Первая строка каждой таблицы — это заголовки столбцов, поэтому по одной ошибке на страницу — это нормально.''''
    
    all_rows = []
    for every_row in initial_table.find_all ('tr'):
        all_elements = every_row.find_all('td')
        try:
            number = all_elements [0].text
            address = all_elements [2].text
            area = all_elements [3].text
            year = all_elements [4].text
            levels = all_elements [5].text
        except:
            print ('Nothing to add')
        
        try:
            link = 'https://dom.mingkh.ru' + every_row.find('a')['href']
            house = soup (requests.get(link).content)

            all_in_table_left = house.find_all ('table')[0].find_all ('td')
            all_in_table_left = [i.text for i in all_in_table_left if i.text != ' ']
            try:
                needs_repair = all_in_table_left[all_in_table_left.index('Дом признан аварийным ')+1]
            except:
                needs_repair = 'Нет информации'
            try:
                raggedness = all_in_table_left [all_in_table_left.index('Состояние дома ')+1]
            except:
                raggedness = 'Нет информации'
            try:
                flat_number = all_in_table_left [all_in_table_left.index('Количество квартир ')+1]
            except:
                flat_number = 'Нет информации'


            all_in_table_right = house.find_all ('table')[1].find_all ('td')
            all_in_table_right = [i.text for i in all_in_table_right if i.text != ' ']
            try:
                house_type = all_in_table_left [all_in_table_left.index('Тип дома ')+1]
            except:
                house_type = 'Нет информации'
                try:
                    house_type = all_in_table_right [all_in_table_right.index('Тип дома ')+1]
                except:
                    house_type = 'Нет информации'
            try:
                living_area = all_in_table_right [all_in_table_right.index('Площадь жилых помещений м2 ')+1]
            except:
                living_area = 'Нет информации'
            try:
                non_living_area = all_in_table_right [all_in_table_right.index('Площадь нежилых помещений м2 ')+1]
            except:
                non_living_area = 'Нет информации'
            try:
                culture = all_in_table_right [all_in_table_right.index('Статус объекта культурного наследия ')+1]
            except:
                culture = 'Нет информации'

            needed_in_table = [needs_repair, raggedness, flat_number, house_type, living_area, non_living_area, culture]

        except:
            print (every_row)
            print ('Nothing to show')
            needed_in_table = ['Нет информации', 'Нет информации', 'Нет информации', 
                               'Нет информации', 'Нет информации', 'Нет информации', 'Нет информации']

        try:
            all_rows_data = [number, address, link, area, year, levels]
            all_rows_data.extend (needed_in_table)
            all_rows.append (all_rows_data)
        except:
            print ('No data')
    first_page = pd.DataFrame (all_rows, columns = ['Номер', 'Адрес', 'Ссылка', 'Площадь всего', 'Год постройки', 'Число этажей', 
                                       'Аварийный', 'Состояние дома', 'Количество квартир', 'Тип дома', 
                                       'Площадь жилых в м2', 'Площадь нежилых в м2', 'Культурное наследие'])
    return first_page

# Шаг 3. Собираем данные

Запускаем код для сбора данных. Обратите внимание, что если ваш компьютер заснёт, код прервёт выполнение. Стоит сохранить накопленный результат в отдельную переменную или файл, продолжить выполнение кода с достигнутой точки, а затем объединить получившиеся файлы. 

In [ ]:
# Шаг 3. Код проходит по всем страницам веб-таблицы города и собирает данные в единую таблицу

general_df = pd.DataFrame ()

for i in tqdm (range (0,329)): # Правым краем диапазона ставим число страниц в таблице города + 1
    
    # Здесь стоит ссылка для Москвы, но вы можете подставить ссылку на другой интересующий вас город
    url = f'https://dom.mingkh.ru/moskva/moskva/houses?page={i}' 
    
    x = soup (requests.get (url).content)
    table = x.find ('table')
    result = get_a_df_from_one_page (table)
    general_df = general_df.append (result, ignore_index=True)

В моём случае выполнение кода прерывалось дважды, поэтому мне пришлось объединить три файла.

In [ ]:
# Шаг 3.1. Объединяем результаты в один датафрейм

df1 = pd.read_csv ('Дома 0_3150.csv').drop ('Unnamed: 0', axis=1)
df2 = pd.read_csv ('Дома 3001_28100.csv').drop ('Unnamed: 0', axis=1)
df3 = pd.read_csv ('Дома 28100_32727.csv').drop ('Unnamed: 0', axis=1)

final_df = df1.append (df2)
final_df = final_df.append (df3)
final_df = final_df.drop_duplicates()
final_df

# Шаг 4. Дособираем данные

У некоторых домов было очень мало информации, поэтому код вернул «Нет информации», хотя она просто была организована иным образом. Также «Нет информации» возвращалось в результате прервавшегося выполнения кода. Чтобы дособрать нужные данные, выделим ссылки на все «несобранные» дома и пройдёмся по ним отдельно. 

In [ ]:
# Шаг 4.1. Выделяем ссылки на дома без информации

all_links_no_info = list (final_df[final_df ['Тип дома']=='Нет информации']['Ссылка'])
all_links_no_info

In [ ]:
# Шаг 4.2. Прописываем функцию для сбора данных по домам без информации

def get_a_df_from_one_page_no_info (link):
    
    ''' Функция принимает на вход ссылку на конкретный дом и возвращает в виде датафрейма всего пять основных параметров:
    1. Ссылка на дом
    2. Признан ли дом аварийным
    3. Состояние дома
    4. Тип дома
    5. Количество квартир
    Остальные поля заполняются как «Нет информации».'''
    
    all_rows = []
    try:
        house = soup (requests.get(link).content)

        all_in_table_left = house.find_all ('table')[0].find_all ('td')
        all_in_table_left = [i.text for i in all_in_table_left if i.text != ' ']
        try:
            needs_repair = all_in_table_left[all_in_table_left.index('Дом признан аварийным ')+1]
        except:
            needs_repair = 'Нет информации'
        try:
            raggedness = all_in_table_left [all_in_table_left.index('Состояние дома ')+1]
        except:
            raggedness = 'Нет информации'
        try:
            house_type = all_in_table_left [all_in_table_left.index('Тип дома ')+1]
        except:
            house_type = 'Нет информации'
        try:
            flat_number = all_in_table_left [all_in_table_left.index('Количество квартир ')+1]
        except:
            flat_number = 'Нет информации'
            

        needed_in_table = [link, needs_repair, raggedness, flat_number, house_type, 'Нет информации', 'Нет информации', 'Нет информации']

    except:
        print ('Nothing to show')
        needed_in_table = ['Нет информации', 'Нет информации', 'Нет информации', 
                           'Нет информации', 'Нет информации', 'Нет информации', 'Нет информации']

    try:
        all_rows.append (needed_in_table)
    except:
        print ('No data')
    first_page = pd.DataFrame (all_rows, columns = ['Ссылка', 'Аварийный', 'Состояние дома', 'Количество квартир', 'Тип дома', 
                                       'Площадь жилых в м2', 'Площадь нежилых в м2', 'Культурное наследие'])
    return first_page

In [ ]:
# Шаг 4.3. Проходимся по ссылкам всех домов без информации и собираем данные об их аварийности и типе здания

key = pd.DataFrame (columns = ['Ссылка', 'Аварийный', 'Состояние дома', 'Количество квартир', 'Тип дома', 
                                       'Площадь жилых в м2', 'Площадь нежилых в м2', 'Культурное наследие'])
for i in tqdm (all_links_no_info):
    may = get_a_df_from_one_page_no_info (i)
    key = key.append (may)
key

Оказалось, что почти все дособранные здания обладают одинаковыми характеристиками: неаварийные, исправные многоквартирные дома. Поэтому мы просто заменяем значения в исходном датафрейме.

In [ ]:
# Шаг 4.4. Заменяем значения

final_df.loc[final_df['Ссылка'].isin(all_links_no_info), 'Аварийный'] = 'Нет'
final_df.loc[final_df['Ссылка'].isin(all_links_no_info), 'Состояние дома'] = 'Исправный'
final_df.loc[final_df['Ссылка'].isin(all_links_no_info), 'Тип дома'] = 'Многоквартирный дом'

Некорректно обработалась только одна ссылка, заменяем данные вручную.

In [ ]:
# Шаг 4.5. Заменяем значения для ссылки-исключения

final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Площадь жилых в м2'] = 5487
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Площадь нежилых в м2'] = 2089
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Культурное наследие'] = 'Нет'
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Количество квартир'] = 108
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860']

# Шаг 5. Подготавливаем адреса для обработки в QGIS

Nominatim OSM распознаёт адреса только в определённом формате, поэтому перед загрузкой в QGIS нам нужно переформатировать адреса с помощью регулярных выражений.

Так, система не понимает «проезд 1-й Тушинский, 3», но понимает «1-й Тушинский проезд, 3».

Адреса с порядковыми номерами и определениями «Малая(ый)/Большая(ой)» унифицировать сложнее. Здесь нужен более сложный алгоритм  с условием «если не найдёшь в Nominatim этот вариант, переделай его по такой схеме».  

Так, в базе дом.минжкх следующие адреса записаны одинаково:
* ул. 1-я Ватутинская, 5
* ул. 1-я Машиностроения, 4

Однако в базе Nominatim OSM они записаны по-разному:
* 1-я Ватутинская ул., 5
* 1-я ул. Машиностроения, 4 к1

Поэтому текущий алгоритм унификации обречён спасать одни адреса и терять другие, потому нуждается в дальнейшей оптимизации.

In [ ]:
# Шаг 5. Унифицируем адреса

import re
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('корпус ', 'к')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('строение ', 'с')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('Строение ', 'с')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('кв-л', 'квартал')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('пр-д', 'проезд')
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^п\. .*, ул\. (.*),(.*)', r'\1 ул.,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^п\. (.*?), (.*)', r'\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^ул\. (.*),(.*)', r'\1 ул.,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^проезд,?\.? (.*),(.*)', r'\1 проезд,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^пер\.,?\.? (.*),(.*)', r'\1 пер,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('пер\.', 'пер')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('б-р', 'бульвар')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('пр-кт', 'проспект')
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^туп\.,?\.? (.*),(.*)', r'\1 туп,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^ш\.,?\.? (.*),(.*)', r'\1 ш,\2', regex=True)

# Шаг 6. Скачиваем данные и геокодируем адреса

Для пакетного геокодирования вам понадобится современная версия QGIS. У меня стоит 3.30.1-'s-Hertogenbosch. Вам нужно зайти в шестерёнку на панели инструментов и найти «Пакетный геокодер Nominatim».

Тем не менее, даже со всеми этапами препроцессинга, алгоритм не распознал или распознал некорректно 22 % адресов. Это вызвано в том числе тем, что в Советском Союзе улицы называли одинаково, так что «Ирония судьбы» актуальна  даже в геоаналитике. 

In [ ]:
# Шаг 6. Скачиваем данные

final_df.to_csv ('Все дома в городе.csv')

Спасибо за внимание! Если есть какие-то вопросы, я всегда доступна:

* Telegram @oilunem
* marikasakowa@gmail.com